In [22]:
from manim import *
import networkx as nx
import numpy as np
import copy

In [23]:
def crea_matriz_imagenes_arbol():
    matriz = [[SVGMobject("imagenes/comida/plato.svg").scale(1.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
               SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
               SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champinon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1), 
                SVGMobject("imagenes/comida/platillos/fusili-pom.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham.svg").scale(.6)],
               [SVGMobject("imagenes/comida/macaroni.svg").rotate(-.2), 
                SVGMobject("imagenes/comida/platillos/macaroni-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/macaroni-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/macaroni-cham.svg").scale(.7)]]
    
    return matriz



In [24]:
def escalar_submatriz(submatriz):
    for i in range(len(submatriz)):
        for j in range(len(submatriz[i])):
            submatriz[i][j].scale(0.5)

def construir_mobject_tabla(comb, primera_columna, primer_renglon):
    return MobjectTable(
        comb,
        row_labels= primera_columna,
        col_labels= primer_renglon,
        include_outer_lines=True
    )

def construir_tabla(self, matriz, tipos_1, tipos_2):
    submatriz = [[matriz[i][j] for j in tipos_2] for i in tipos_1]
    escalar_submatriz(submatriz)
    primera_columna = [submatriz[i][0] for i in range(1, len(submatriz))]
    primer_renglon = submatriz[0][1:]
    comb = [row[1:] for row in submatriz[1:]]
    tabla = construir_mobject_tabla(comb, primera_columna, primer_renglon)
    return tabla


In [25]:
def distribuir_puntos_uniformemente(n, dist):
    puntos = []
    if n % 2 == 0:  # Si n es par, se divide niformemente a ambos lados de 0.
        cant_puntos_lado = n // 2
        puntos_lado = [dist * (i - 1/2) for i in range(1, cant_puntos_lado + 1)]
    else:  # Si n es impar, se coloca un punto en 0 y se divide el resto uniformemente.
        cant_puntos_lado = (n - 1) // 2
        puntos_lado = [dist * i for i in range(1, cant_puntos_lado + 1)]
        puntos += [0]
    
    # Distribuye puntos uniformemente arriba y abajo de 0 con una distancia de 0.5.
    
    puntos.extend(puntos_lado)
    puntos.extend(-1* np.array(puntos_lado))

    return sorted(puntos, reverse=True)

In [26]:
def generar_coordenadas_y_aristas_vertical(puntos_previos, coordenadas_finales, divisiones, x, distancia):
    # Almacena las coordenadas y las aristas del árbol
    coordenadas = [coordenadas_finales]
    aristas = []
    
    for dec in reversed(divisiones):
        nueva_capa = []
        nuevos_puntos = []
        
        # Mueve el nivel de decisión hacia la izquierda
        x -= distancia

        for i in range(0, len(puntos_previos), dec):
            # Calcula el valor y para el punto actual
            valor_y = round((puntos_previos[i] + puntos_previos[i + dec - 1]) / 2, 2)
            nueva_capa.append((x, valor_y, 0))
            nuevos_puntos.append(valor_y)
            
            # Agrega las aristas entre el nivel actual y el nivel siguiente
            for j in range(i, i + dec):
                arista = [(x, -valor_y, -5), (x + distancia, -puntos_previos[j], -5)]
                aristas.insert(0, arista)
        
        # Agrega las coordenadas de la capa actual y actualiza los puntos previos
        coordenadas.insert(0, nueva_capa)
        puntos_previos = nuevos_puntos
    
    return coordenadas, aristas

def distribucion_vertical(divisiones, total_combinaciones, distancia_y=0.8):
    # Calcula los valores iniciales
    niveles = len(divisiones)
    distancia_x = 2
    x_inicial = niveles * distancia_x - 6
    
    # Distribuye uniformemente los puntos en el último nivel del arbol
    puntos_iniciales = distribuir_puntos_uniformemente(total_combinaciones, distancia_y)
    coordenadas_iniciales = [(x_inicial, valor, 0) for valor in puntos_iniciales]
    
    # Genera todas las coordenadas y aristas para cada nivel
    coordenadas, aristas = generar_coordenadas_y_aristas_vertical(
        puntos_iniciales, coordenadas_iniciales, divisiones, x_inicial, distancia_x
    )
    
    return coordenadas, aristas

In [27]:
def generar_coordenadas_y_aristas_horizontal(puntos_previos, coordenadas_finales, divisiones, y, distancia):
    # Almacena las coordenadas y aristas para el árbol de decisión horizontal
    coordenadas = [coordenadas_finales]
    aristas = []
    
    for dec in reversed(divisiones):
        nueva_capa = []
        nuevos_puntos = []
        
        # Mueve el nivel de decisión hacia arriba
        y += distancia

        for i in range(0, len(puntos_previos), dec):
            # Calcula el valor x para el punto actual
            valor_x = round((puntos_previos[i] + puntos_previos[i + dec - 1]) / 2, 2)
            nueva_capa.append((valor_x, y, 0))
            nuevos_puntos.append(valor_x)
            
            # Agrega las aristas entre el nivel actual y el siguiente
            for j in range(i, i + dec):
                arista = [(-valor_x, y, -5), (-puntos_previos[j], y - distancia, -5)]
                aristas.insert(0, arista)
        
        # Agrega las coordenadas de la capa actual y actualiza los puntos previos
        coordenadas.insert(0, nueva_capa)
        puntos_previos = nuevos_puntos
    
    return coordenadas, aristas

def distribucion_horizontal(divisiones, total_combinaciones, distancia_x=None):
    # Configura la distancia entre puntos
    if distancia_x is None:
        distancia_x = 12 / total_combinaciones
    
    niveles = len(divisiones)
    distancia_y = 1.5 if niveles <= 4 else 1
    y_inicial = 3 - niveles * distancia_y
    
    # Genera la capa inicial de puntos distribuidos uniformemente
    puntos_iniciales = distribuir_puntos_uniformemente(total_combinaciones, distancia_x)
    coordenadas_iniciales = [(valor, y_inicial, 0) for valor in puntos_iniciales]
    
    # Genera todas las coordenadas y aristas para cada nivel
    coordenadas, aristas = generar_coordenadas_y_aristas_horizontal(
        puntos_iniciales, coordenadas_iniciales, divisiones, y_inicial, distancia_y
    )
    
    return coordenadas, aristas


In [39]:
def elegir_orientacion_y_coordenadas(decisiones_niveles, dist=0.8):
    """
    Elige orientación vertical u horizontal para las coordenadas.
    """
    total_combinaciones = np.prod(decisiones_niveles)
    if total_combinaciones <= 10:
        return distribucion_vertical(decisiones_niveles, total_combinaciones, dist)
    else:
        return distribucion_horizontal(decisiones_niveles, total_combinaciones, dist)

In [29]:
def transponer_matriz(matriz):
    transpuesta = [[0] * len(matriz) for _ in range(len(matrix[0]))]

    for i in range(len(matriz)):
        for j in range(len(matriz[0])):
            transpuesta[j][i] = matriz[i][j]

    return transpuesta

In [38]:
def generar_combinaciones(matriz_completa, tipos_1, tipos_2):
    """
    Genera una lista de combinaciones de acuerdo con las filas y columnas indicadas.
    """
    submatriz = [[matriz_completa[i][j] for j in tipos_2] for i in tipos_1]
    tipos_2 = submatriz[0][1:]
    combinaciones = [fila[0] for fila in submatriz]
    
    for i in range(1, len(tipos_1)):
        copias_tipos_2 =[copy.deepcopy(combinacion) for combinacion in tipos_2]
        combinaciones.extend(copias_tipos_2)
    
    for j in range(1, len(tipos_1)):
        combinaciones.extend(submatriz[j][1:])
    
    return combinaciones

In [41]:
def construir_arbol(matriz, filas, columnas, dist=0.8):
    # Genera el número de decisiones por nivel
    decisiones_por_nivel = obtener_decisiones_nivel(filas, columnas)
    
    # Elige orientación (vertical/horizontal) y obtiene las coordenadas de los objetos y aristas
    coordenadas, aristas = elegir_orientacion_y_coordenadas(decisiones_por_nivel, dist)
    
    # Genera las combinaciones de la matriz de datos
    combinaciones = generar_combinaciones(matriz, filas, columnas)
    
    # Construye los niveles del árbol y organiza las aristas
    arbol_niveles, arbol, aristas = construir_niveles_arbol(coordenadas, combinaciones, 
                                                            aristas, 
                                                            decisiones_por_nivel)

    return arbol_niveles, arbol, coordenadas, aristas

def obtener_decisiones_nivel(filas, columnas):
    """
    Genera la lista de decisiones en cada nivel basado en el tamaño de filas y columnas.
    """
    return [len(filas) - 1, len(columnas) - 1, 1]

def construir_niveles_arbol(coordenadas, combinaciones, aristas, niveles_decision):
    arbol_niveles = []
    arbol = VGroup()
    indice_combinacion = 0
    j = 0
    nivel_actual = 0
    num_aristas = 1
    
    # Recorre cada nivel de coordenadas
    for piso in coordenadas:
        nivel = VGroup()
        
        # Añade cada combinación en su posición correspondiente
        for punto in piso:
            escala = 0.3 if nivel_actual != 1 else 0.4
            combinaciones[indice_combinacion].move_to(punto).scale(escala)
            arbol.add(combinaciones[indice_combinacion])
            nivel.add(combinaciones[indice_combinacion])
            indice_combinacion +=1 
        
        # Añade aristas entre niveles
        if nivel_actual < len(niveles_decision):
            aristas, num_aristas, nivel_actual, j = agregar_aristas(
                niveles_decision, aristas, num_aristas, arbol, nivel, nivel_actual, j
            )

        arbol_niveles.append(nivel)
        nivel_actual += 1
        
    return arbol_niveles, arbol, aristas

def agregar_aristas(valores, aristas, num_aristas, arbol, nivel, nivel_actual, j):
    num_aristas *= valores[nivel_actual]
    indice_arista = 0
    
    # Genera las aristas entre los niveles
    while j < len(aristas) and indice_arista < num_aristas:
        linea = Line(start = aristas[j][0], end = aristas[j][1])
        linea.z_index = -5
        arbol.add(linea)
        nivel.add(linea)
        j += 1
        indice_arista += 1
    return aristas, num_aristas, nivel_actual, j


def construir_niveles_arbol_2(coordenadas, combinaciones, aristas, niveles_decision):
    arbol_niveles = []
    arbol = VGroup()
    indice_combinacion = 0
    nivel_actual = 0

    # Recorre cada nivel de coordenadas
    for nivel_coordenadas in coordenadas:
        nivel = VGroup()
        
        # Añade cada combinación en su posición correspondiente
        for punto in nivel_coordenadas:
            escala = 0.3 if nivel_actual != 1 else 0.4
            combinaciones[indice_combinacion].move_to(punto).scale(escala)
            arbol.add(combinaciones[indice_combinacion])
            nivel.add(combinaciones[indice_combinacion])
            indice_combinacion += 1

        # Añade aristas entre niveles
        if nivel_actual < len(niveles_decision):
            aristas, nivel_actual = crear_aristas(niveles_decision, aristas, arbol, nivel, nivel_actual)

        arbol_niveles.append(nivel)
        nivel_actual += 1
    
    return arbol_niveles, arbol, aristas

def crear_aristas_2(niveles_decision, aristas, arbol, nivel, nivel_actual):
    cantidad_aristas = 1 * niveles_decision[nivel_actual]
    indice_arista = 0

    # Genera las aristas entre los niveles
    while indice_arista < len(aristas) and indice_arista < cantidad_aristas:
        linea = Line(start=aristas[indice_arista][0], end=aristas[indice_arista][1], z_index=-5)
        arbol.add(linea)
        nivel.add(linea)
        indice_arista += 1

    return aristas, nivel_actual



In [32]:
def crear_arbol_niveles(self,arbol_niveles):
    for obj in arbol_niveles:
        group = VGroup(*obj)
        self.add(group)
        self.play(FadeIn(group))

In [33]:
def imagenes_arbol_complicado():
    matriz = [[[SVGMobject("imagenes/comida/plato.svg").scale(1.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
               SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
               SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champinon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1), 
                SVGMobject("imagenes/comida/platillos/fusili-pom.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham.svg").scale(.6)]],
             [[SVGMobject("imagenes/comida/salmon.svg").scale(.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
              SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
              SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champinon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo-salmon.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto-salmon.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham-salmon.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1),
                SVGMobject("imagenes/comida/platillos/fusili-pom-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo-salmon.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto-salmon.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham-salmon.svg").scale(.6)]],
             [[SVGMobject("imagenes/comida/carne.svg").scale(.7), SVGMobject("imagenes/comida/salsa_pomodoro.svg"), 
              SVGMobject("imagenes/comida/salsa_parmesano.svg"), SVGMobject("imagenes/comida/salsa_ajo.svg"), 
              SVGMobject("imagenes/comida/salsa_pesto.svg"), SVGMobject("imagenes/comida/salsa_champinon.svg")],
               [SVGMobject("imagenes/comida/espagueti.svg").scale(1.8).rotate(-1), 
                SVGMobject("imagenes/comida/platillos/espagueti-pom-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-par-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-ajo-carne.svg").scale(.7), 
                SVGMobject("imagenes/comida/platillos/espagueti-pesto-carne.svg").scale(.7),
                SVGMobject("imagenes/comida/platillos/espagueti-cham-carne.svg").scale(.7)],
               [SVGMobject("imagenes/comida/fusili.svg").rotate(-.1),
                SVGMobject("imagenes/comida/platillos/fusili-pom-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-par-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-ajo-carne.svg").scale(.6), 
                SVGMobject("imagenes/comida/platillos/fusili-pesto-carne.svg").scale(.6),
                SVGMobject("imagenes/comida/platillos/fusili-cham-carne.svg").scale(.6)]]]
    return matriz


In [34]:
def comb_pasta(matriz_completa, tipos_1, tipos_2, tipos_3):
    submatriz = [[[matriz_completa[i][j][k] for k in tipos_2] for j in tipos_1] for i in tipos_3]
    n = len(tipos_3) -1
    p = len(tipos_2) -1
    k = len(tipos_1) -1
    valores = [k, p, n,1]

    tipos_1, tipos_2, tipos_3 = ajustar_tipos(submatriz)

    combinaciones = [matriz_completa[0][0][0]]
    combinaciones.extend(tipos_1)

    combinaciones = crear_matriz_copias(tipos_1, tipos_2, tipos_3, submatriz, combinaciones, k , p ,n)
    
    return combinaciones, valores

def ajustar_tipos(submatriz):
    tipos_1 = [submatriz[0][i][0] for i in range(1, len(submatriz[0]))]
    
    tipos_2 = submatriz[0][0][1:]

    tipos_3 = [submatriz[i][0][0] for i in range(1, len(submatriz))]
    
    return tipos_1, tipos_2, tipos_3
    
def crear_matriz_copias(tipos_1, tipos_2, tipos_3, submatriz, combinaciones, k , p ,n):

    for i in range(0, len(tipos_1)):
        copias_tipos2 =copy.deepcopy(tipos_2)
        combinaciones.extend(copias_tipos2)

    for i in range(0, len(tipos_1) * len(tipos_2)):
        copias_tipos3 = copy.deepcopy(tipos_3)
        combinaciones.extend(copias_tipos3)

    for i in range(0, k):
        for r in range(0, p):
            for j in range(0,n):
                copia_tipos3 = copy.deepcopy(submatriz[j+1][(i+1)][r+1])
                combinaciones.append(copia_tipos3)
    return combinaciones

In [35]:
def construir_arbol_compl(matriz_completa, tipos_1, tipos_2, tipos_3, dist = .8):
    dec1 = len(tipos_1) - 1
    dec2 = len(tipos_2) - 1
    dec3 = len(tipos_3) - 1
    dec = [dec1,dec2,dec3,1]
    combinaciones, valores = comb_pasta(matriz_completa, tipos_1, tipos_2, tipos_3)
    coordenadas, aristas = elegir_orientacion_y_obtener_coordenadas(valores, dist)
    
    arbol_niveles, arbol, aristas = construir_pisos_arbol_compl(coordenadas, combinaciones, aristas, valores)
    

    return (arbol_niveles, arbol, coordenadas, aristas)

def construir_pisos_arbol_compl(coordenadas, combinaciones, aristas, dec):
    arbol_niveles = []
    arbol = VGroup()
    i = 0
    j=0
    k = 0
    num_aristas = 1
    for piso in coordenadas:
        nivel = VGroup()
        for punto in piso:
            escala = 0.25 if k != 1 else 0.3
            combinaciones[i].move_to(punto).scale(escala)
            arbol.add(combinaciones[i])
            nivel.add(combinaciones[i])
            i +=1 
        if k < len(dec):
            num_aristas *= dec[k]
            r = 0
            while j < len(aristas) and r < num_aristas:
                linea = Line(start = aristas[j][0], end = aristas[j][1])
                linea.z_index = -5
                #self.play(Write(edge))
                arbol.add(linea)
                nivel.add(linea)
                j += 1
                r += 1
            
        arbol_niveles.append(nivel)
        k += 1
    return arbol_niveles, arbol, aristas

In [36]:
def animar_arbol_sencillo(self, fusilli, salsa_pom, salsa_par, salsa_ajo):
    """
    Reacomoda los objetos con las imagenes de las salsas y de la pasta fusili y crea lineas entre ellos para convertirlo 
    en un arbol de decision. Luego crea imagenes con el platillo resultante de cada camino y los une con lineas al arbol.

    Args:
        fusili (SVGMobject): Objeto con la imagen de la pasta fusili 
        salsa_pom, salsa_par, salsa_ajo (SVGMobject): Objetos con imagenes de las pastas
    
    Return:
        grupo_arbol (VGroup): vector que agrupa todos los objetos visuales del arbol rendereado

    """

    self.play(fusilli.animate.move_to([-5, 0, 0]), 
            salsa_pom.animate.move_to([1, -2.5, 0]).scale(.7),
            salsa_par.animate.move_to([1, 0, 0]).scale(.7),
            salsa_ajo.animate.move_to([1, 2.5, 0]).scale(.7), 
            run_time = 1
            )
    
    lineas_a_salsas = _crear_lineas_de_fusilli_a_salsas(self, fusilli, salsa_pom, salsa_par, salsa_ajo)
    
    # Crear imágenes con los platillos resultantes de cada camino del arbol
    platillos = _crear_platillos(self, )

    lineas_a_platillos = _crear_lineas_a_platillos(self, salsa_pom, salsa_par, salsa_ajo, *platillos)
    
    # Agrupar todos los elementos del árbol
    grupo_arbol = VGroup(fusilli, salsa_pom, salsa_par, salsa_ajo, *platillos,
            *lineas_a_salsas, *lineas_a_platillos)
    
    return grupo_arbol

def _crear_lineas_de_fusilli_a_salsas(self, fusilli, salsa_pomodoro, salsa_parmesano, salsa_ajo):
    lineas = [
        Line(start=fusilli.get_center(), end=salsa.get_center()).set_z_index(-1)
        for salsa in (salsa_pomodoro, salsa_parmesano, salsa_ajo)
    ]
    self.play(*(Create(linea) for linea in lineas))
    return lineas

def _crear_platillos(self, ):
    fus_pom = SVGMobject("imagenes/comida/platillos/fusili-pom.svg").move_to([4.5, -2.5, 0]).scale(.5)
    fus_par = SVGMobject("imagenes/comida/platillos/fusili-par.svg").move_to([4.5, 0, 0]).scale(.5)
    fus_ajo = SVGMobject("imagenes/comida/platillos/fusili-ajo.svg").move_to([4.5, 2.5, 0]).scale(.5)
    self.play(Write(fus_pom), Write(fus_par),Write(fus_ajo),  run_time=.9)
    
    return fus_pom, fus_par, fus_ajo

def _crear_lineas_a_platillos(self, salsa_pomodoro, salsa_parmesano, salsa_ajo, plato_pomodoro, plato_parmesano, plato_ajo):
    salsas = [salsa_pomodoro, salsa_parmesano, salsa_ajo]
    platillos = [plato_pomodoro, plato_parmesano, plato_ajo]
    lineas = [
        Line(start=salsa.get_center(), end=platillo.get_center()).set_z_index(-1)
        for salsa, platillo in zip(salsas, platillos)
    ]
    self.play(*(Create(linea) for linea in lineas), tiempo_ejecucion=.5)
    return lineas